# Import thư viện

In [1]:
#import các thư viện 
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
keras.__version__

'2.9.0'

# Load data + Pre-processing

In [3]:
# MNIST là mảng 2 phần tử, phần tử đầu chứa train set, phần tử sau chứa test set.
# Train set và test set là mảng 2 phần tử, phần tử đầu chứa data features, phần tử sau chứa data labels.
# Train set sẽ 60000 data features tương ứng với 60000 labels, test set có 10000.
# Mỗi data feature là mảng 28 chiều, mỗi phần tử lại chứa một mảng 28 chiều. Đây đại diện cho một bức ảnh 28x28.

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Có 60000 ảnh dùng để train và valid, 10000 ảnh dùng để test")
print("Mỗi ảnh có một kênh màu, kích thước 28x28")

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
Có 60000 ảnh dùng để train và valid, 10000 ảnh dùng để test
Mỗi ảnh có một kênh màu, kích thước 28x28


In [ ]:
# Rescale vào đoạn [0,1] để giữ weights không quá lớn. Mỗi pixel ảnh có giá trị từ 0 đến 255, do đó ta chia 255.
X_train, X_test = X_train/255.0, X_test/255.0

#Tách một phần tập train thành tập valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1) # Tách theo tỉ lệ validation/train=1/9

# Model

## Add Layers

In [ ]:
inputs = keras.layers.Input(shape=(28,28))
inputs_reshape = keras.layers.Reshape((28,28,1))(inputs)

conv_1 = keras.layers.Convolution2D(filters=20,
                                 kernel_size=[5,5],
                                 strides=[1,1],
                                 padding='same',
                                 activation=tf.nn.relu)(inputs_reshape)
maxpool_1 = keras.layers.MaxPool2D(pool_size=[2,2],
                                strides=[2,2])(conv_1)

conv_2 = keras.layers.Convolution2D(filters=20,
                                 kernel_size=[5,5],
                                 strides=[1,1],
                                 padding='valid',
                                 activation=tf.nn.relu)(maxpool_1)
maxpool_2 = keras.layers.MaxPool2D(pool_size=[2,2],
                                strides=[2,2])(conv_2)

flatten = keras.layers.Flatten()(maxpool_2)
softmax_1 = keras.layers.Dense(units=20, activation='sigmoid')(flatten)
softmax_2 = keras.layers.Dense(units=10, activation='softmax')(softmax_1)

## Compile

In [ ]:
model = keras.models.Model(inputs=inputs, outputs=softmax_2)

model.compile(optimizer='adam',
             loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 20)        10020     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 20)         0         
 2D)                                                         

In [ ]:
### Cách tính các kết quả có trong Param #

## Convolutional Layer: ((kernel_width * kernel_height * số_kênh_của_output_trước) + 1(bias)) * số_filter
# conv2d(filters=20, kernel_size=5x5, stride=1): (((5 * 5 * 1) + 1) * 20) = 520
# conv2d_1(filters=20, kernel_size=5x5, stride=1): (((5 * 5 * 20) + 1) * 20) = 10020

## Fully Connected Layer: Số node ở layer hiện tại * Số node ở layer trước + 1(bias) * Số node ở layer hiện tại
# dense: 20 * 500 + 1 * 20 = 10020
# dense_1: 10 * 20 + 1 * 10 = 210

## Training

In [ ]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=100,
          validation_data=(X_valid,y_valid))

Epoch 1/10
540/540 [==============================] - 52s 94ms/step - loss: 0.7972 - accuracy: 0.8460 - val_loss: 0.2934 - val_accuracy: 0.9545
Epoch 2/10
540/540 [==============================] - 51s 95ms/step - loss: 0.1999 - accuracy: 0.9646 - val_loss: 0.1465 - val_accuracy: 0.9690
Epoch 3/10
540/540 [==============================] - 50s 93ms/step - loss: 0.1196 - accuracy: 0.9746 - val_loss: 0.1044 - val_accuracy: 0.9777
Epoch 4/10
540/540 [==============================] - 50s 93ms/step - loss: 0.0857 - accuracy: 0.9804 - val_loss: 0.0804 - val_accuracy: 0.9805
Epoch 5/10
540/540 [==============================] - 51s 95ms/step - loss: 0.0667 - accuracy: 0.9837 - val_loss: 0.0628 - val_accuracy: 0.9852
Epoch 6/10
540/540 [==============================] - 50s 93ms/step - loss: 0.0552 - accuracy: 0.9866 - val_loss: 0.0562 - val_accuracy: 0.9865
Epoch 7/10
540/540 [==============================] - 52s 96ms/step - loss: 0.0465 - accuracy: 0.9881 - val_loss: 0.0502 - val_accuracy:

## Testing

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Loss trên tập test: ", test_loss," Accuracy trên tập test: ", test_acc)

313/313 [==============================] - 4s 11ms/step - loss: 0.0426 - accuracy: 0.9869
Loss trên tập test:  0.0426153764128685  Accuracy trên tập test:  0.9868999719619751
